In [1]:
# wrap the output in colab cells
from IPython.display import HTML, display

def set_css():
  display(HTML('''

  '''))
get_ipython().events.register('pre_run_cell', set_css)


In [2]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.2 MB/s eta 0:00:00


In [5]:
file = open("/content/main text.txt", "r")
FileContent = file.read().strip()

In [6]:
FileContent

'" Welcome to this new course on Principles of Compiler Design. So, in this lecture I will give you an overview of a compiler, but before that we will also see how exactly the course is organized and then the motivation for studying compiler design and of course, then go on to the details of a compiler with block diagrams. So, the course is actually a first level course. In other words, this takes a detail look at the internals of a compiler and I do not assume any background for this particular course, but it is a very intensive course. So, the pace of the course is going to be you know not very slow, but at the same time it is not going to be Ray C either, but the students who are actually going to take this course seriously are requested to do the programming assignments. They are also supposed to solve theoretical problems, which I am going to suggest. Otherwise, this course will not be understood you know properly. The reason is a compiler is an excellent example of the theory bei

In [7]:
len(FileContent)

39952

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

checkpoint = "sshleifer/distilbart-cnn-12-6"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [9]:
tokenizer.model_max_length

1024

In [10]:
tokenizer.max_len_single_sentence

1022

In [11]:
tokenizer.num_special_tokens_to_add()

2

Converting File content to Sentences

In [12]:
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [13]:
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])

78

Create The Chunks

In [14]:
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter

  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter

    # if it is the last sentence
    if count == len(sentences) - 1:
      chunks.append(chunk.strip()) # save the chunk

  else:
    chunks.append(chunk.strip()) # save the chunk

    # reset
    length = 0
    chunk = ""

    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

8

In [15]:
[len(tokenizer.tokenize(c)) for c in chunks]

[1019, 995, 1007, 994, 1014, 1001, 991, 828]

In [16]:
[len(tokenizer(c).input_ids) for c in chunks]

[1021, 997, 1009, 996, 1016, 1003, 993, 830]

With Special Tokens Added

In [17]:
sum([len(tokenizer(c).input_ids) for c in chunks])

7865

In [18]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (7851 > 1024). Running this sequence through the model will result in indexing errors


7851

Without Special Tokens Added

In [19]:
len(tokenizer(FileContent).input_ids)

7851

In [20]:
len(tokenizer.tokenize(FileContent))

7849

Get The Inputs

In [21]:
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]

In [ ]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output, skip_special_tokens=True))

 A compiler is an excellent example of the theory being translated into practice and this is a wonderful example of that. Compilers are used in software engineering as well as software testing and then program optimization then in the security domain malicious code detection design of new computer architectures. Compiler technology is used to generate a simulator and the simulator is actually a computer program which is generated for that particular program.
